# Hyperparameter Tuning for Phishing Detection Models

Optuna-based hyperparameter optimization for:
- TF-IDF + Logistic Regression
- XGBoost Hybrid
- MLP Hybrid

**Objective:** Maximize Recall + ROC-AUC (catch phishing + good ranking)

## Setup and Imports

In [2]:
import os
import sys
import json
import pickle
from datetime import datetime
from pathlib import Path
import numpy as np
import polars as pl
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
     precision_score, recall_score, f1_score, roc_auc_score,
)
from sentence_transformers import SentenceTransformer
import xgboost as xgb
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback

sys.path.append(str(Path.cwd().parent.parent))

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
print(f"Optuna version: {optuna.__version__}")

PyTorch version: 2.5.1+cu121
CUDA available: True
Device: cuda
Optuna version: 4.6.0


## Configuration

In [17]:
config = {
    "random_seed": 42,
    "data_source": "../../emails_v6.parquet",
    "numeric_features": [
        "sender_domain_entropy", "has_attachment",
        "spf_flag_missing", "dkim_flag_missing",
        "num_links", "subject_length", "body_length", "keyword_count", 
        "num_received_headers", "num_exclamation_marks", 
        "num_malicious_links"
    ],
    "embedding_model": "all-MiniLM-L6-v2",
    "embedding_dim": 384,
    "n_trials_tfidf": 30,
    "n_trials_xgb": 30,
    "n_trials_mlp": 30
}

In [4]:
# Load dataset
print("Loading dataset...")
df = pl.read_parquet("../../emails_v6.parquet")
pl.Config.set_tbl_rows(-1)
print(df.select("source").unique())
print(f"Total records: {len(df):,}")
print(f"Columns: {df.columns}")

# Check class distribution
phishing_count = df.filter(pl.col("phishing") == 1).shape[0]
legit_count = df.filter(pl.col("phishing") == 0).shape[0]

print(f"\nClass distribution:")
print(f"  Phishing: {phishing_count:,} ({phishing_count/len(df)*100:.2f}%)")
print(f"  Legitimate: {legit_count:,} ({legit_count/len(df)*100:.2f}%)")


from app.data_processing.preprocessing_pipeline import PreprocessingPipeline

df = PreprocessingPipeline().process_pipeline(df)
external_df = df.filter(
    (pl.col("source") == "data/csv_misc/TREC-07.csv") | (pl.col("source") == "phishing-2020")
)

train_pool_df = df.filter(
    (pl.col("source") != "data/csv_misc/TREC-07.csv") & (pl.col("source") != "phishing-2020")
)

print(f"Training pool: {len(train_pool_df):,}")
print(f"External validation: {len(external_df):,}")

train_pool_pd = train_pool_df.to_pandas()

train_df_pd, test_df_pd = train_test_split(
    train_pool_pd, 
    test_size=0.25, 
    random_state=config["random_seed"], 
    stratify=train_pool_pd['phishing']
)

train_df = pl.from_pandas(train_df_pd)
test_df = pl.from_pandas(test_df_pd)

print(f"\nFinal splits:")
print(f"  Train: {len(train_df):,} ({len(train_df)/len(df)*100:.1f}%)")
print(f"  Test: {len(test_df):,} ({len(test_df)/len(df)*100:.1f}%)")
print(f"  External: {len(external_df):,} ({len(external_df)/len(df)*100:.1f}%)")
numeric_features = config["numeric_features"]


features_train = train_df.select(numeric_features).fill_null(0).to_numpy()
features_test = test_df.select(numeric_features).fill_null(0).to_numpy()
y_train = train_df['phishing'].to_numpy()
y_test = test_df['phishing'].to_numpy()


Loading dataset...
shape: (22, 1)
┌────────────────────────────┐
│ source                     │
│ ---                        │
│ str                        │
╞════════════════════════════╡
│ phishing-2013              │
│ phishing-2024              │
│ mbox_andrunik.mbox         │
│ data/csv_misc/TREC-06.csv  │
│ phishing-2015              │
│ phishing-nigerian          │
│ phishing-2018              │
│ phishing-2014              │
│ mbox_gradzki.mbox          │
│ phishing-2016              │
│ phishing-2020              │
│ phishing-2021              │
│ data/csv_misc/Enron.csv    │
│ data/csv_misc/Assassin.csv │
│ data/csv_misc/TREC-07.csv  │
│ phishing-2023              │
│ phishing-2019              │
│ phishing-2017              │
│ data/csv_misc/CEAS-08.csv  │
│ data/csv_misc/Ling.csv     │
│ data/csv_misc/TREC-05.csv  │
│ phishing-2022              │
└────────────────────────────┘
Total records: 212,113
Columns: ['source', 'phishing', 'spf_flag', 'dkim_flag', 'd_flag', 'num_rec

## Generate Embeddings

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = SentenceTransformer(config["embedding_model"], device=device)

print("Encoding train set...")
X_emb_train = embedding_model.encode(
    train_df['body_subject'].to_list(), 
    show_progress_bar=True, 
    convert_to_numpy=True,
    batch_size=32
)

print("Encoding test set...")
X_emb_test = embedding_model.encode(
    test_df['body_subject'].to_list(), 
    show_progress_bar=True, 
    convert_to_numpy=True,
    batch_size=32
)

print(f"Embeddings shape: {X_emb_train.shape}")

Encoding train set...


Batches: 100%|██████████| 3723/3723 [02:36<00:00, 23.77it/s] 


Encoding test set...


Batches: 100%|██████████| 1241/1241 [00:56<00:00, 22.15it/s] 


Embeddings shape: (119121, 384)


## TF-IDF Tuning

In [6]:
from scipy.sparse import hstack
def tfidf_objective(trial):
    params = {
        'max_features': trial.suggest_int('max_features', 3000, 10000, step=1000),
        'min_df': trial.suggest_int('min_df', 2, 10),
        'max_df': trial.suggest_float('max_df', 0.5, 0.95, step=0.05),
        'ngram_max': trial.suggest_int('ngram_max', 1, 3),
        'C': trial.suggest_float('C', 0.1, 10.0, log=True),
        'class_weight_ratio': trial.suggest_float('class_weight_ratio', 1.0, 10.0, step=0.5),
        'threshold': trial.suggest_float('threshold', 0.2, 0.5, step=0.05)
    }
    
    vectorizer = TfidfVectorizer(
        max_features=params['max_features'],
        ngram_range=(1, params['ngram_max']),
        min_df=params['min_df'],
        max_df=params['max_df']
    )
    
    X_tfidf_train = vectorizer.fit_transform(train_df['body_subject'].to_list())
    X_tfidf_test = vectorizer.transform(test_df['body_subject'].to_list())
    
    X_combined_train = hstack([X_tfidf_train, features_train])
    X_combined_test = hstack([X_tfidf_test, features_test])
    
    clf = LogisticRegression(
        max_iter=1000,
        C=params['C'],
        class_weight={0: 1.0, 1: params['class_weight_ratio']},
        random_state=42,
        verbose=0
    )
    clf.fit(X_combined_train, y_train)
    
    y_proba = clf.predict_proba(X_combined_test)[:, 1]
    y_pred = (y_proba >= params['threshold']).astype(int)
    
    recall = recall_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_proba)
    
    trial.set_user_attr('recall', recall)
    trial.set_user_attr('precision', precision)
    trial.set_user_attr('f1_score', f1)
    trial.set_user_attr('roc_auc', roc_auc)
    F2 = (5 * precision * recall) / (4 * precision + recall)
    return F2

In [7]:
print("Starting TF-IDF tuning...")

wandb_callback = WeightsAndBiasesCallback(
    metric_name="objective_score",
    wandb_kwargs={"project": "phishstop-detection", "tags": ["optuna", "tfidf"]}
)

tfidf_study = optuna.create_study(
    direction="maximize",
    study_name="tfidf-tuning",
    sampler=optuna.samplers.TPESampler(seed=42)
)

tfidf_study.optimize(
    tfidf_objective,
    n_trials=config["n_trials_tfidf"],
    callbacks=[wandb_callback],
    show_progress_bar=True
)

print(f"\nBest score: {tfidf_study.best_trial.value:.4f}")
print(f"Recall: {tfidf_study.best_trial.user_attrs['recall']:.4f}")
print(f"ROC-AUC: {tfidf_study.best_trial.user_attrs['roc_auc']:.4f}")
print(f"Precision: {tfidf_study.best_trial.user_attrs['precision']:.4f}")
print(f"F1: {tfidf_study.best_trial.user_attrs['f1_score']:.4f}")
print("\nBest params:")
for k, v in tfidf_study.best_trial.params.items():
    print(f"  {k}: {v}")

C:\Users\Latandu\AppData\Local\Temp\ipykernel_20948\287095509.py:3: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandb_callback = WeightsAndBiasesCallback(


Starting TF-IDF tuning...


wandb: Currently logged in as: latandu (latandu-phishstop) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[I 2025-11-30 22:12:12,202] A new study created in memory with name: tfidf-tuning
Best trial: 0. Best value: 0.951436:   3%|▎         | 1/30 [00:58<28:14, 58.43s/it]

[I 2025-11-30 22:13:10,619] Trial 0 finished with value: 0.9514355174808982 and parameters: {'max_features': 5000, 'min_df': 10, 'max_df': 0.8500000000000001, 'ngram_max': 2, 'C': 0.20513382630874505, 'class_weight_ratio': 2.0, 'threshold': 0.2}. Best is trial 0 with value: 0.9514355174808982.


...

[I 2025-11-30 23:00:34,864] Trial 27 finished with value: 0.97542122108823 and parameters: {'max_features': 7000, 'min_df': 3, 'max_df': 0.65, 'ngram_max': 1, 'C': 2.1450357089870633, 'class_weight_ratio': 1.0, 'threshold': 0.45}. Best is trial 19 with value: 0.9799043062200957.


Best trial: 19. Best value: 0.979904:  97%|█████████▋| 29/30 [49:29<01:00, 60.21s/it]

[I 2025-11-30 23:01:41,460] Trial 28 finished with value: 0.9792705232578989 and parameters: {'max_features': 8000, 'min_df': 6, 'max_df': 0.55, 'ngram_max': 2, 'C': 3.226677116219035, 'class_weight_ratio': 4.0, 'threshold': 0.4}. Best is trial 19 with value: 0.9799043062200957.


Best trial: 19. Best value: 0.979904: 100%|██████████| 30/30 [50:38<00:00, 101.29s/it]

[I 2025-11-30 23:02:50,927] Trial 29 finished with value: 0.9769762548842803 and parameters: {'max_features': 5000, 'min_df': 4, 'max_df': 0.65, 'ngram_max': 2, 'C': 5.9857926374935815, 'class_weight_ratio': 2.5, 'threshold': 0.5}. Best is trial 19 with value: 0.9799043062200957.

Best score: 0.9799
Recall: 0.9918
ROC-AUC: 0.9974
Precision: 0.9352
F1: 0.9626

Best params:
  max_features: 8000
  min_df: 6
  max_df: 0.6
  ngram_max: 2
  C: 4.842855693488871
  class_weight_ratio: 4.5
  threshold: 0.4


In [14]:
fig = optuna.visualization.plot_optimization_history(tfidf_study)
fig.update_layout(width=800, height=500) 
fig.show()

fig = optuna.visualization.plot_param_importances(tfidf_study)
fig.update_layout(width=800, height=500) 
fig.show()

## XGBoost Tuning

In [9]:
def xgboost_objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500, step=50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0, step=0.05),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0, step=0.5),
        'threshold': trial.suggest_float('threshold', 0.2, 0.5, step=0.05)
    }
    
    X_train = np.concatenate([X_emb_train, features_train], axis=1)
    X_test = np.concatenate([X_emb_test, features_test], axis=1)
    
    clf = xgb.XGBClassifier(
        max_depth=params['max_depth'],
        n_estimators=params['n_estimators'],
        learning_rate=params['learning_rate'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        scale_pos_weight=params['scale_pos_weight'],
        random_state=42,
        eval_metric='logloss',
        early_stopping_rounds=10,
        verbosity=0
    )
    
    clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    
    y_proba = clf.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= params['threshold']).astype(int)
    
    recall = recall_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_proba)
    
    trial.set_user_attr('recall', recall)
    trial.set_user_attr('precision', precision)
    trial.set_user_attr('f1_score', f1)
    trial.set_user_attr('roc_auc', roc_auc)
    
    F2 = (5 * precision * recall) / (4 * precision + recall)
    return F2   

In [10]:
print("Starting XGBoost tuning...")

wandb_callback_xgb = WeightsAndBiasesCallback(
    metric_name="objective_score",
    wandb_kwargs={"project": "phishstop-detection", "tags": ["optuna", "xgboost"]}
)

xgb_study = optuna.create_study(
    direction="maximize",
    study_name="xgboost-tuning",
    sampler=optuna.samplers.TPESampler(seed=42)
)

xgb_study.optimize(
    xgboost_objective,
    n_trials=config["n_trials_xgb"],
    callbacks=[wandb_callback_xgb],
    show_progress_bar=True
)

print(f"\nBest score: {xgb_study.best_trial.value:.4f}")
print(f"Recall: {xgb_study.best_trial.user_attrs['recall']:.4f}")
print(f"ROC-AUC: {xgb_study.best_trial.user_attrs['roc_auc']:.4f}")
print(f"Precision: {xgb_study.best_trial.user_attrs['precision']:.4f}")
print(f"F1: {xgb_study.best_trial.user_attrs['f1_score']:.4f}")
print("\nBest params:")
for k, v in xgb_study.best_trial.params.items():
    print(f"  {k}: {v}")

Starting XGBoost tuning...


C:\Users\Latandu\AppData\Local\Temp\ipykernel_20948\757620496.py:3: ExperimentalWarning:

WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.



C,▁▇▁▄▁▁▁▅▁▄▂▂▂▇▆█▃▇▄▄▃▄▅▅▃▂▅▂▃▅
class_weight_ratio,▂▇▃▂█▂▅▅▃▁▇▆▄▆▆▆▅▆▅▄▄▅▃▃▃▃▄▁▃▂
max_df,▆▆▃▃▆▁▃▆▁▅█▁█▄▅▄▃▄▃▃▂▃▃▃▂▂▃▃▂▃
max_features,▃▇▂▂▅▇▁▂▁▃█▇▆███▅▆▆▆▆▄▅▅▄▅▄▅▆▃
min_df,█▅▂▃▁▃██▂▃▅▅▅▆▆▆▇▆▅▅▄▅▄▄▄▄▄▂▅▃
ngram_max,▅▁▅▅▁█▅█▁▁▁███▅▅█▅▅▅▅▅▅▅▅▅▅▁▅▅
objective_score,▁▇▆▇▄▇▄▇▆▇▅▆▆▇▇█▇████▇█████▇█▇
threshold,▁▂▆▄█▄▄█▇█▁▂▃▃▂▃▃▃▆▆▆▆▇▇▇▇▇▇▆█
C,5.98579
class_weight_ratio,2.5
max_df,0.65


[I 2025-11-30 23:02:55,336] A new study created in memory with name: xgboost-tuning
Best trial: 0. Best value: 0.977166:   3%|▎         | 1/30 [00:44<21:44, 44.98s/it]

[I 2025-11-30 23:03:40,308] Trial 0 finished with value: 0.9771659644534842 and parameters: {'max_depth': 6, 'n_estimators': 500, 'learning_rate': 0.1205712628744377, 'min_child_weight': 6, 'subsample': 0.65, 'colsample_bytree': 0.65, 'scale_pos_weight': 1.5, 'threshold': 0.5}. Best is trial 0 with value: 0.9771659644534842.


...

[I 2025-11-30 23:24:57,313] Trial 27 finished with value: 0.979115582988078 and parameters: {'max_depth': 6, 'n_estimators': 400, 'learning_rate': 0.15183422653050652, 'min_child_weight': 3, 'subsample': 0.9, 'colsample_bytree': 0.75, 'scale_pos_weight': 3.0, 'threshold': 0.45}. Best is trial 25 with value: 0.9813506606230109.


Best trial: 25. Best value: 0.981351:  97%|█████████▋| 29/30 [22:39<00:47, 47.50s/it]

[I 2025-11-30 23:25:34,948] Trial 28 finished with value: 0.979746040942449 and parameters: {'max_depth': 7, 'n_estimators': 400, 'learning_rate': 0.2102934635913294, 'min_child_weight': 5, 'subsample': 0.8, 'colsample_bytree': 0.8, 'scale_pos_weight': 5.5, 'threshold': 0.45}. Best is trial 25 with value: 0.9813506606230109.


Best trial: 25. Best value: 0.981351: 100%|██████████| 30/30 [23:17<00:00, 46.59s/it]

[I 2025-11-30 23:26:13,105] Trial 29 finished with value: 0.9787431017277242 and parameters: {'max_depth': 5, 'n_estimators': 500, 'learning_rate': 0.1022268832486803, 'min_child_weight': 3, 'subsample': 0.75, 'colsample_bytree': 0.7, 'scale_pos_weight': 6.0, 'threshold': 0.5}. Best is trial 25 with value: 0.9813506606230109.

Best score: 0.9814
Recall: 0.9855
ROC-AUC: 0.9979
Precision: 0.9650
F1: 0.9751

Best params:
  max_depth: 7
  n_estimators: 400
  learning_rate: 0.08981654015231944
  min_child_weight: 1
  subsample: 0.9
  colsample_bytree: 0.8
  scale_pos_weight: 5.5
  threshold: 0.45


In [11]:
fig = optuna.visualization.plot_optimization_history(xgb_study)
fig.update_layout(width=800, height=500) 
fig.show()

fig = optuna.visualization.plot_param_importances(xgb_study)
fig.update_layout(width=800, height=500) 
fig.show()

## MLP Tuning

In [18]:
from hybrid_mlp_model import HybridMLPClassifier

def mlp_objective(trial):
    params = {
        'feature_hidden_dim': trial.suggest_categorical('feature_hidden_dim', [16, 32, 64, 128]),
        'dropout': trial.suggest_float('dropout', 0.1, 0.5, step=0.1),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128]),
        'pos_weight': trial.suggest_float('pos_weight', 1.0, 10.0, step=0.5),
        'threshold': trial.suggest_float('threshold', 0.2, 0.5, step=0.05)
    }
    
    X_emb_train_fit, X_emb_val, X_feat_train_fit, X_feat_val, y_train_fit, y_val = train_test_split(
        X_emb_train, features_train, y_train,
        test_size=0.2,
        random_state=42,
        stratify=y_train
    )
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = HybridMLPClassifier(
        embedding_dim=config["embedding_dim"],
        num_features=len(numeric_features),
        feature_hidden_dim=params['feature_hidden_dim'],
        dropout=params['dropout']
    ).to(device)
    
    train_loader = DataLoader(
        TensorDataset(
            torch.FloatTensor(X_emb_train_fit),
            torch.FloatTensor(X_feat_train_fit),
            torch.FloatTensor(y_train_fit)
        ),
        batch_size=params['batch_size'],
        shuffle=True
    )
    
    val_loader = DataLoader(
        TensorDataset(
            torch.FloatTensor(X_emb_val),
            torch.FloatTensor(X_feat_val),
            torch.FloatTensor(y_val)
        ),
        batch_size=params['batch_size'],
        shuffle=False
    )
    
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=params['lr'],
        weight_decay=params['weight_decay']
    )
    
    criterion = nn.BCEWithLogitsLoss(
        pos_weight=torch.tensor([params['pos_weight']], device=device)
    )
    
    best_val_loss = float('inf')
    patience = 5
    patience_counter = 0
    
    for epoch in range(30):
        model.train()
        for embeddings, features, labels in train_loader:
            embeddings, features, labels = embeddings.to(device), features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            logits = model(embeddings, features).squeeze()
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for embeddings, features, labels in val_loader:
                embeddings, features, labels = embeddings.to(device), features.to(device), labels.to(device)
                logits = model(embeddings, features).squeeze()
                val_loss += criterion(logits, labels).item()
        
        val_loss /= len(val_loader)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break
    
    test_loader = DataLoader(
        TensorDataset(
            torch.FloatTensor(X_emb_test),
            torch.FloatTensor(features_test),
            torch.FloatTensor(y_test)
        ),
        batch_size=params['batch_size'],
        shuffle=False
    )
    
    model.eval()
    test_probs = []
    test_labels = []
    
    with torch.no_grad():
        for embeddings, features, labels in test_loader:
            embeddings, features = embeddings.to(device), features.to(device)
            logits = model(embeddings, features).squeeze()
            probs = torch.sigmoid(logits)
            
            if probs.dim() == 0:
                test_probs.append(probs.cpu().item())
                test_labels.append(labels.item())
            else:
                test_probs.extend(probs.cpu().numpy())
                test_labels.extend(labels.numpy())
    
    test_probs = np.array(test_probs)
    test_labels = np.array(test_labels)
    test_preds = (test_probs >= params['threshold']).astype(int)
    
    recall = recall_score(test_labels, test_preds, zero_division=0)
    precision = precision_score(test_labels, test_preds, zero_division=0)
    f1 = f1_score(test_labels, test_preds, zero_division=0)
    roc_auc = roc_auc_score(test_labels, test_probs)
    
    trial.set_user_attr('recall', recall)
    trial.set_user_attr('precision', precision)
    trial.set_user_attr('f1_score', f1)
    trial.set_user_attr('roc_auc', roc_auc)
    
    F2 = (5 * precision * recall) / (4 * precision + recall)
    return F2

In [19]:
print("Starting MLP tuning...")

wandb_callback_mlp = WeightsAndBiasesCallback(
    metric_name="objective_score",
    wandb_kwargs={"project": "phishstop-detection", "tags": ["optuna", "mlp"]}
)

mlp_study = optuna.create_study(
    direction="maximize",
    study_name="mlp-tuning",
    sampler=optuna.samplers.TPESampler(seed=42)
)

mlp_study.optimize(
    mlp_objective,
    n_trials=config["n_trials_mlp"],
    callbacks=[wandb_callback_mlp],
    show_progress_bar=True
)

print(f"\nBest score: {mlp_study.best_trial.value:.4f}")
print(f"Recall: {mlp_study.best_trial.user_attrs['recall']:.4f}")
print(f"ROC-AUC: {mlp_study.best_trial.user_attrs['roc_auc']:.4f}")
print(f"Precision: {mlp_study.best_trial.user_attrs['precision']:.4f}")
print(f"F1: {mlp_study.best_trial.user_attrs['f1_score']:.4f}")
print("\nBest params:")
for k, v in mlp_study.best_trial.params.items():
    print(f"  {k}: {v}")

Starting MLP tuning...


C:\Users\Latandu\AppData\Local\Temp\ipykernel_20948\2423746168.py:3: ExperimentalWarning:

WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.



[I 2025-11-30 23:31:23,044] A new study created in memory with name: mlp-tuning
Best trial: 0. Best value: 0.979567:   3%|▎         | 1/30 [01:33<45:02, 93.17s/it]

[I 2025-11-30 23:32:56,212] Trial 0 finished with value: 0.9795673804364932 and parameters: {'feature_hidden_dim': 32, 'dropout': 0.1, 'lr': 0.00020511104188433984, 'weight_decay': 1.3066739238053272e-05, 'batch_size': 32, 'pos_weight': 1.0, 'threshold': 0.5}. Best is trial 0 with value: 0.9795673804364932.


Best trial: 1. Best value: 0.98136:   7%|▋         | 2/30 [02:17<30:01, 64.35s/it] 

...

[I 2025-12-01 00:10:31,844] Trial 27 finished with value: 0.9823456597243085 and parameters: {'feature_hidden_dim': 128, 'dropout': 0.30000000000000004, 'lr': 0.001525543829793603, 'weight_decay': 1.5698388730328185e-05, 'batch_size': 128, 'pos_weight': 7.0, 'threshold': 0.4}. Best is trial 16 with value: 0.9830404281478268.


Best trial: 16. Best value: 0.98304:  97%|█████████▋| 29/30 [40:26<01:02, 62.65s/it]

[I 2025-12-01 00:11:49,449] Trial 28 finished with value: 0.9805543566872873 and parameters: {'feature_hidden_dim': 128, 'dropout': 0.5, 'lr': 0.00010288381788614386, 'weight_decay': 3.378313289367936e-05, 'batch_size': 128, 'pos_weight': 4.0, 'threshold': 0.30000000000000004}. Best is trial 16 with value: 0.9830404281478268.


Best trial: 16. Best value: 0.98304: 100%|██████████| 30/30 [41:33<00:00, 83.12s/it]

[I 2025-12-01 00:12:56,763] Trial 29 finished with value: 0.9813573397200591 and parameters: {'feature_hidden_dim': 16, 'dropout': 0.4, 'lr': 0.00017920349900006513, 'weight_decay': 1.0197122484200126e-05, 'batch_size': 64, 'pos_weight': 5.5, 'threshold': 0.5}. Best is trial 16 with value: 0.9830404281478268.

Best score: 0.9830
Recall: 0.9873
ROC-AUC: 0.9981
Precision: 0.9662
F1: 0.9766

Best params:
  feature_hidden_dim: 128
  dropout: 0.30000000000000004
  lr: 0.0002540994280961815
  weight_decay: 0.0003358499007747041
  batch_size: 128
  pos_weight: 5.0
  threshold: 0.45


In [20]:
fig = optuna.visualization.plot_optimization_history(mlp_study)
fig.update_layout(width=800, height=500) 
fig.show()

fig = optuna.visualization.plot_param_importances(mlp_study)
fig.update_layout(width=800, height=500) 
fig.show()

## Save Results

In [21]:
results = {
    'tfidf': {
        'best_score': tfidf_study.best_trial.value,
        'recall': tfidf_study.best_trial.user_attrs['recall'],
        'roc_auc': tfidf_study.best_trial.user_attrs['roc_auc'],
        'precision': tfidf_study.best_trial.user_attrs['precision'],
        'f1_score': tfidf_study.best_trial.user_attrs['f1_score'],
        'best_params': tfidf_study.best_trial.params
    },
    'xgboost': {
        'best_score': xgb_study.best_trial.value,
        'recall': xgb_study.best_trial.user_attrs['recall'],
        'roc_auc': xgb_study.best_trial.user_attrs['roc_auc'],
        'precision': xgb_study.best_trial.user_attrs['precision'],
        'f1_score': xgb_study.best_trial.user_attrs['f1_score'],
        'best_params': xgb_study.best_trial.params
    },
    'mlp': {
        'best_score': mlp_study.best_trial.value,
        'recall': mlp_study.best_trial.user_attrs['recall'],
        'roc_auc': mlp_study.best_trial.user_attrs['roc_auc'],
        'precision': mlp_study.best_trial.user_attrs['precision'],
        'f1_score': mlp_study.best_trial.user_attrs['f1_score'],
        'best_params': mlp_study.best_trial.params
    }
}

os.makedirs("../../output/tuning_results", exist_ok=True)

with open('../../output/tuning_results/optuna_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Results saved to output/tuning_results/optuna_results.json")
print("\nSummary:")
for model, res in results.items():
    print(f"\n{model.upper()}:")
    print(f"  Score F2 = (5 * precision * recall) / (4 * precision + recall): {res['best_score']:.4f}")
    print(f"  Recall: {res['recall']:.4f}")
    print(f"  ROC-AUC: {res['roc_auc']:.4f}")
    print(f"  Precision: {res['precision']:.4f}")
    print(f"  F1 Score: {res['f1_score']:.4f}")

Results saved to output/tuning_results/optuna_results.json

Summary:

TFIDF:
  Score F2 = (5 * precision * recall) / (4 * precision + recall): 0.9799
  Recall: 0.9918
  ROC-AUC: 0.9974
  Precision: 0.9352
  F1 Score: 0.9626

XGBOOST:
  Score F2 = (5 * precision * recall) / (4 * precision + recall): 0.9814
  Recall: 0.9855
  ROC-AUC: 0.9979
  Precision: 0.9650
  F1 Score: 0.9751

MLP:
  Score F2 = (5 * precision * recall) / (4 * precision + recall): 0.9830
  Recall: 0.9873
  ROC-AUC: 0.9981
  Precision: 0.9662
  F1 Score: 0.9766
